# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](../static_files/autograd-variable.png)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [1]:
import torch

Create a tensor

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` was created as a result of an operation, so it has a `grad_fn`.

In [6]:
print(y.grad_fn)

Do more operations on y

In [7]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor

In [9]:
# 1
t = torch.rand(5,5,requires_grad=True)
print(t)
# 2
sum_t = torch.sum(t) # thought to be a funtion of t
print(sum_t)
print(sum_t.grad_fn)

tensor([[0.0044, 0.9690, 0.8872, 0.7973, 0.7184],
        [0.6325, 0.4267, 0.7403, 0.9793, 0.4630],
        [0.3186, 0.3560, 0.9200, 0.7786, 0.2586],
        [0.9650, 0.3053, 0.5396, 0.7329, 0.0277],
        [0.6643, 0.4879, 0.5730, 0.1656, 0.2928]], requires_grad=True)
tensor(14.0039, grad_fn=<SumBackward0>)


## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`

In [10]:
out.backward()

Print gradients d(out)/dx

In [11]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [12]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 246.1212, 1572.9935, -399.9373], grad_fn=<MulBackward0>)


In [13]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


**Read later** \
*Documentation* \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor and set `requires_grad` to `True`
3. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
7. Print all gradients again
  - what did you notice?

In [31]:
# 1
t = torch.rand(1,1,requires_grad=True)
print('t: ', t)

# 2
t1 = t.mul(2)
print('t1: ', t1)

# 3
t_sum = t1.sum()
print('t_sum: ', t_sum)

# 4
print('t_grad: ', t.grad)
print('t1_grad: ', t1.grad)
print('t_sum_grad: ', t_sum.grad)

# 5
t1.retain_grad()
t_sum.retain_grad()
t_sum.backward()

# 6
# y(f(x))
print('t_grad: ', t.grad) #dy/dx 
print('t1_grad: ', t1.grad) # dy/df
print('t_sum_grad: ', t_sum.grad) # dy/dy

t:  tensor([[0.7151]], requires_grad=True)
t1:  tensor([[1.4302]], grad_fn=<MulBackward0>)
t_sum:  tensor(1.4302, grad_fn=<SumBackward0>)
t_grad:  None
t1_grad:  None
t_sum_grad:  None
t_grad:  tensor([[2.]])
t1_grad:  tensor([[1.]])
t_sum_grad:  tensor(1.)
